In [66]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [19]:

def load_images_and_labels(directories, exclude_dir=None):
    labels = []
    images = []
    for directory in directories:
        for dir_name, _, file_list in os.walk(directory):
            if dir_name == exclude_dir:  # skip the excluded directory
                continue
            for filename in file_list:
                if filename.endswith('.png'):
                    # the label is the first word of the filename
                    label = filename.split(' ')[0]  # update the separator depending on your file naming convention
                    labels.append(label)

                    # load and preprocess the image
                    img_path = os.path.join(dir_name, filename)
                    img = load_img(img_path, target_size=(32, 32))  # the target size depends on your requirements
                    img_array = img_to_array(img) / 255.0  # convert to array and normalize to range [0, 1]
                    images.append(img_array)
    
    print(f"Number of images: {len(images)}, Number of labels: {len(labels)}")  # Debug line

    if not images:
        print(f"No images found in directories: {directories}")

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    if labels:
        labels = le.fit_transform(labels)
        labels = tf.keras.utils.to_categorical(labels)
    else:
        print("No labels found. Returning...")
        return [], []

    return images, labels


In [20]:
train_dirs = ["./data/Fold1",
              "./data/Fold2",
              "./data/Fold3",
              "./data/Fold4"]
test_dir = "./data/Fold5"

train_images, train_labels = load_images_and_labels(train_dirs, exclude_dir=test_dir)
test_images, test_labels = load_images_and_labels([test_dir])


Number of images: 200, Number of labels: 200
Number of images: 50, Number of labels: 50


In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

def build_model():
    model = Sequential()

    # Layer 1: Convolutional + Average pooling
    model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(32, 32, 3)))
    model.add(AveragePooling2D())

    # Layer 2: Convolutional + Average pooling
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'))
    model.add(AveragePooling2D())

    # Flatten the image to one dimensional array
    model.add(Flatten())

    # Layer 3: Fully connected layer (Dense)
    model.add(Dense(units=120, activation='tanh'))

    # Layer 4: Fully connected layer (Dense)
    model.add(Dense(units=84, activation='tanh'))

    # Layer 5: Output layer
    model.add(Dense(units=5, activation = 'softmax'))

    return model

In [83]:
models_accuracy = []

In [84]:
from tensorflow.keras.optimizers import Adam

# Specify training parameters: batch size and number of epochs
batch_size = 32
epochs = 20

def train_model(train_images, train_labels, verbose):
    # Building model
    model= build_model()
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Importing data

    # Convert images and labels to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)

    # Train the model using the training data
    history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=verbose)
    return model

In [85]:
def test_model(model, test_images, test_labels):
    # Convert test images and labels to numpy arrays
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(test_images, test_labels)

    print(f"Test set accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [86]:
train_dirs = ["./data/Fold1", "./data/Fold2", "./data/Fold3", "./data/Fold4"]
test_dir = "./data/Fold5"

train_images, train_labels = load_images_and_labels(train_dirs, exclude_dir=test_dir)
test_images, test_labels = load_images_and_labels([test_dir])

model = train_model(train_images, train_labels, verbose=0)

accuracy = test_model(model, test_images, test_labels)
models_accuracy.append(accuracy)

Number of images: 200, Number of labels: 200
Number of images: 50, Number of labels: 50
2/2 [==============================] - 0s 6ms/step - loss: 0.1345 - accuracy: 0.9800
Test set accuracy: 98.00%


In [87]:
train_dirs = ["./data/Fold5", "./data/Fold1", "./data/Fold2", "./data/Fold3"]
test_dir = "./data/Fold4"

train_images, train_labels = load_images_and_labels(train_dirs, exclude_dir=test_dir)
test_images, test_labels = load_images_and_labels([test_dir])

model = train_model(train_images, train_labels, verbose=0)

accuracy = test_model(model, test_images, test_labels)
models_accuracy.append(accuracy)

Number of images: 200, Number of labels: 200
Number of images: 50, Number of labels: 50
2/2 [==============================] - 0s 8ms/step - loss: 0.0868 - accuracy: 1.0000
Test set accuracy: 100.00%


In [88]:
train_dirs = ["./data/Fold4", "./data/Fold5", "./data/Fold1", "./data/Fold2"]
test_dir = "./data/Fold3"

train_images, train_labels = load_images_and_labels(train_dirs, exclude_dir=test_dir)
test_images, test_labels = load_images_and_labels([test_dir])

model = train_model(train_images, train_labels, verbose=0)

accuracy = test_model(model, test_images, test_labels)
models_accuracy.append(accuracy)

Number of images: 200, Number of labels: 200
Number of images: 50, Number of labels: 50
2/2 [==============================] - 0s 9ms/step - loss: 0.1395 - accuracy: 0.9600
Test set accuracy: 96.00%


In [89]:
train_dirs = ["./data/Fold3", "./data/Fold4", "./data/Fold5", "./data/Fold1"]
test_dir = "./data/Fold2"

train_images, train_labels = load_images_and_labels(train_dirs, exclude_dir=test_dir)
test_images, test_labels = load_images_and_labels([test_dir])

model = train_model(train_images, train_labels, verbose=0)

accuracy = test_model(model, test_images, test_labels)
models_accuracy.append(accuracy)

Number of images: 200, Number of labels: 200
Number of images: 50, Number of labels: 50
2/2 [==============================] - 0s 7ms/step - loss: 0.0927 - accuracy: 0.9400
Test set accuracy: 94.00%


In [90]:
train_dirs = ["./data/Fold2", "./data/Fold3", "./data/Fold4", "./data/Fold5"]
test_dir = "./data/Fold1"

train_images, train_labels = load_images_and_labels(train_dirs, exclude_dir=test_dir)
test_images, test_labels = load_images_and_labels([test_dir])

model = train_model(train_images, train_labels, verbose=0)

accuracy = test_model(model, test_images, test_labels)
models_accuracy.append(accuracy)

Number of images: 200, Number of labels: 200
Number of images: 50, Number of labels: 50
2/2 [==============================] - 0s 5ms/step - loss: 0.2374 - accuracy: 0.8800
Test set accuracy: 88.00%


In [99]:
mean_accuracy = np.average(models_accuracy)
std_accuracy = np.std(models_accuracy)
print(f"Acurácia média: {mean_accuracy * 100:.2f}%")
print(f"Desvio Padrão: {std_accuracy* 100:.2f}%")


Acurácia média: 95.20%
Desvio Padrão: 4.12%
